In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments


# Fine-Tune the university Chatbot Model for better Responses 
## Preprocessing the data
### Using intents dataset from kaagle 
https://www.kaggle.com/datasets/tusharpaul2001/university-chatbot-dataset?resource=download


In [2]:
import json

# Load the intents.json file
with open("intents.json", "r") as file:
    data = json.load(file)


# Extract the list of intents
intents = data['intents']
#print(intents)

### seprating the json data to responses and questions 

In [3]:
questions = []
responses = []

for intent in intents:
    for text in intent["text"]:
        questions.append(text)
    for response in intent["responses"]:
        responses.append(response)




In [4]:
# merge back the dataset 
dataset = list(zip(questions, responses))

In [5]:
#formating data
formatted_data = [{"input": q, "output": r} for q, r in dataset]

In [6]:
#writing into the file
import json

with open("dataset.jsonl", "w") as file:
    for item in formatted_data:
        file.write(json.dumps(item) + "\n")

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Load the dataset
from datasets import load_dataset
dataset = load_dataset("json", data_files="dataset.jsonl")

# Tokenize the dataset
def preprocess_function(examples):
    inputs = [q for q in examples["input"]]
    targets = [r for r in examples["output"]]
    
    # Tokenize inputs and targets with padding and truncation
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    
    # Add labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
    
tokenized_dataset = dataset.map(preprocess_function, batched=True)



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [10]:
# Debug: Check lengths (truncate error)
for example in tokenized_dataset["train"]:
    print(f"Input IDs length: {len(example['input_ids'])}, Labels length: {len(example['labels'])}")


Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels length: 128
Input IDs length: 128, Labels leng

## Training The model 

In [11]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

# Fine-tune the model
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


TrainOutput(global_step=21, training_loss=29.35618663969494, metrics={'train_runtime': 470.3378, 'train_samples_per_second': 0.319, 'train_steps_per_second': 0.045, 'total_flos': 25678400716800.0, 'train_loss': 29.35618663969494, 'epoch': 3.0})

### Saving the model

In [12]:
model.save_pretrained("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-model")

('./fine-tuned-model\\tokenizer_config.json',
 './fine-tuned-model\\special_tokens_map.json',
 './fine-tuned-model\\spiece.model',
 './fine-tuned-model\\added_tokens.json')

### UI stuff using gradio

In [20]:
conversation_history = []

def generate_response(user_input, history):
    # Combine history and current input
    full_input = " ".join(history + [user_input])
    input_ids = tokenizer(full_input, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    history.append(user_input)
    history.append(response)
    return response, history

def chat():
    print("Welcome to the University Customer Support Chatbot! Type 'exit' to end the chat.")
    history = []
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break
        response, history = generate_response(user_input, history)
        print(f"Chatbot: {response}")

In [21]:
# Run the chat
if __name__ == "__main__":
    chat()

Welcome to the University Customer Support Chatbot! Type 'exit' to end the chat.


You:  hey


Chatbot: hey hey hey hey


You:   What are the admission requirements?


Chatbot: a high school diploma or equivalent


You:  exit


Chatbot: Goodbye!


In [22]:
import gradio as gr

# Function to generate responses
def generate_response(user_input):
    try:
        # Add context to the input
        prompt = "Answer the following question about university admissions: "
        input_text = prompt + user_input

        # Tokenize the input
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids

        # Generate the output
        outputs = model.generate(
            input_ids,
            max_length=128,
            num_beams=5,
            early_stopping=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Fallback for short or generic responses
        if len(response.split()) < 5:
            response = "I'm sorry, I couldn't find a detailed answer. Please visit our admissions page or contact the admissions office for more information."

        return response
    except Exception as e:
        return "An error occurred. Please try again later."

# Gradio chat interface function
def respond(message, history):
    response = generate_response(message)
    return response

# Create a Gradio ChatInterface
interface = gr.ChatInterface(
    fn=respond,
    title="University Customer Support Chatbot",
    description="Ask me anything about admissions, courses, or campus life!",
    examples=[
        "What are the admission requirements?",
        "Do I need to submit SAT scores for admission?",
        "What is the minimum GPA required for admission?"
    ]
)

# Launch the interface
interface.launch()

C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\gradio\components\chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
